In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import lmfit


In [2]:
data = pd.read_csv('GCN4_fourfield_R2.tsv', sep='\t')
data = data.loc[(data.resi==5)&(data.field<17.0)].reset_index(drop=True)

In [3]:
def exponential_decay(par, xdata):
    
    # Parse multiple input parameter
    # formats for intensity, rate    
    if hasattr(par,'valuesdict'):
        # lmfit parameter format
        var = par.valuesdict()
        inten = var['inten']
        rate = var['rate']
    elif hasattr(par,'keys'):
        # dict format
        inten = par['inten']
        rate = par['rate']
    else:
        # array/list/tuple format
        inten = par[0]
        rate = par[1]

    # Calculate the y-data from the parameters
    return inten * np.exp(-1*rate*xdata)

In [4]:
func = exponential_decay
groupcols = ['field']
xname = 'time'
yname = 'intensity'

method = 'leastsq'
sigma = 0.95
threads = 4

params = [{'name':'inten', 'value':np.asarray(data.groupby(groupcols)[yname].max()), 'vary':True},
          {'name':'rate', 'value':20.0, 'vary':True}]

In [5]:
# pdlmfit __init__

input_data = data
func = func
groupcols = groupcols
params = params
xname = xname
yname = yname
yerr = None
method = method
sigma = sigma
threads = threads

index = ( data[groupcols + [xname]]
          .groupby(groupcols)
          .max()
          .index
         )
ngroups = index.shape[0]

In [7]:
def convert_param_dict_to_df(params, ngroups, index):

    # Unique list of variable names
    var_names = map(lambda x: x['name'], params)

    # Expanded list of names and all properties to create a 
    # multi-level column index
    column_list = [(x['name'], y) 
                   for x in params 
                       for y in x.keys()
                  ]

    column_index = pd.MultiIndex.from_tuples(column_list, 
                                             sortorder=None)

    # Create a dataframe from the indexes and fill it
    param_df = pd.DataFrame(index=index, columns=column_index)

    # Fill df by iterating over the parameter name index
    # and then each of its keys
    for var in enumerate(param_df.columns.levels[0]):
        for key in param_df.loc[:,var[1]]:
            param_df[(var[1],key)] = params[var[0]][key]
            
    param_df.fillna(method='ffill', inplace=True)

    return param_df

def convert_param_df_to_expanded_list(param_df):
    ### CLEAN ###

    # Convert each parameter entry to a list of dictionaries
    list_of_dicts = [ [ param_df.loc[x,y].to_dict() 
                        for y in param_df.columns.levels[0]] 
                      for x in param_df.index]

    # Convert the list of dictionaries to a list of lmfit parameters
    param_list = [ [ lmfit.Parameter(**r) 
                     for r in row ] 
                  for row in list_of_dicts ]

    return param_list

In [9]:
# Working copy of pdlmfit.py fit function

if isinstance(params[0], dict):
    param_df = convert_param_dict_to_df(params, ngroups, index)
# TODO: otherwise check if this is a dataframe
    
params = convert_param_df_to_expanded_list(param_df)
paramnames = [[x.name for x in row] for row in params]

In [10]:
paramnames

[['inten', 'rate'], ['inten', 'rate']]

[[<Parameter 'inten', 15647.389418999999, bounds=[-inf:inf]>,
  <Parameter 'rate', 20.0, bounds=[-inf:inf]>],
 [<Parameter 'inten', 68415.828125, bounds=[-inf:inf]>,
  <Parameter 'rate', 20.0, bounds=[-inf:inf]>]]

In [40]:
param_df.loc[:,('inten')].to_dict(orient='records')

[{'name': 'inten', 'value': 15647.389419, 'vary': True},
 {'name': 'inten', 'value': 68415.828125, 'vary': True}]

In [23]:
series_to_dict

[[{('inten', 'name'): 'inten',
   ('inten', 'value'): 15647.389419,
   ('inten', 'vary'): True},
  {('rate', 'name'): 'rate', ('rate', 'value'): 20.0, ('rate', 'vary'): True}],
 [{('inten', 'name'): 'inten',
   ('inten', 'value'): 68415.828125,
   ('inten', 'vary'): True},
  {('rate', 'name'): 'rate', ('rate', 'value'): 20.0, ('rate', 'vary'): True}]]

In [22]:
param_list

[[<Parameter 'inten', 15647.389419, bounds=[-inf:inf]>,
  <Parameter 'rate', 20.0, bounds=[-inf:inf]>],
 [<Parameter 'inten', 68415.828125, bounds=[-inf:inf]>,
  <Parameter 'rate', 20.0, bounds=[-inf:inf]>]]

In [24]:
param_df

inten                       rate            
       vary   name         value  vary  name value
field                                             
14.1   True  inten  15647.389419  True  rate  20.0
16.5   True  inten  68415.828125  True  rate  20.0

In [38]:
name_list

['inteninteninten', 'raterateraterate']

In [33]:
# pdlmfit fit
column_index.values

array([('inten', 'vary'), ('inten', 'name'), ('inten', 'value'),
       ('rate', 'a'), ('rate', 'vary'), ('rate', 'name'), ('rate', 'value')], dtype=object)

In [36]:
name_list

['inten', 'inten', 'inten', 'rate', 'rate', 'rate', 'rate']